In [1]:
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import time

In [2]:
def cal_distance(x, y):
    #return torch.sum((x-y)**2)**0.5
    #return torch.sum((x-y)**4)**0.5
    #return torch.sum((x-y)**6)**0.5 #best
    #return 1-torch.cosine_similarity(x,y,dim=0) #95.5 k=11
    # return torch.dist(x, y, 6)#best k=6
    return torch.dist(x, y, 2)
    

In [3]:

# iteration func
def KNN_by_iter(train_x, train_y, test_x, test_y, k):
    pca=PCA(0.9)
    pca.fit(train_x)
    train_x_reduct=torch.from_numpy(pca.transform(train_x))
    test_x_reduct=torch.from_numpy(pca.transform(test_x))
    
    since = time.time()

    # cal distance
    res = []
    for x in tqdm(test_x_reduct):
        dists = []
        for y in train_x_reduct:
            dists.append(cal_distance(x,y).view(1))
        
        idxs = torch.cat(dists).argsort()[:k]
        res.append(np.bincount(np.array([train_y[idx] for idx in idxs])).argmax())

    print(res[:10])#?
    print("acc",accuracy_score(test_y, res))

    time_elapsed = time.time() - since
    print('KNN iter training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        

In [6]:
def gaussian_noise(x, severity=1):
    """Gaussian noise corruption to images.
    Args:
    x: numpy array, uncorrupted image, assumed to have uint8 pixel in [0,1].
    severity: integer, severity of corruption.
    Returns:
    numpy array, image with pixels in [0,1]. Added Gaussian noise.
    """
    c = [.08, .12, 0.18, 0.26, 0.38][severity - 1]
    x=np.array(x)
    x_clip = np.clip(x + np.random.normal(size=x.shape, scale=c), 0, 1)
    return x_clip

In [7]:
def Run_Knn(train_dataset,gauss_para,test_x,test_y):
    train_x = []
    train_y = []
    
    for i in range(len(train_dataset)):
    # index = [i for i in range(len(test_dataset))] 
    # random.shuffle(index)
    # for i in index[0:10000]:
        img, target = train_dataset[i]
        img=img.squeeze()
        img=torch.from_numpy(gaussian_noise(img,gauss_para))
        
        # plt.imshow(img,'gray')
        # plt.title(target)
        # plt.show()
        # break
        
        train_x.append(img.view(-1))
        train_y.append(target)
        if i > 10000:
            break
        
    KNN_by_iter(torch.stack(train_x), train_y, torch.stack(test_x), test_y, 4)

In [8]:
if __name__ == "__main__":

    train_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=True,download=True)
    test_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=False,download=True)

    test_x = [] 
    test_y = []
    for i in range(len(test_dataset)):
        img, target = test_dataset[i]
        img=img.squeeze()      
        test_x.append(img.view(-1))
        test_y.append(target)


    # print("classes:" , set(train_y))
    for gauss_para in range(2,6):
        Run_Knn(train_dataset,gauss_para,test_x,test_y)
    


100%|██████████| 10000/10000 [36:31<00:00,  4.56it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9438
KNN iter training complete in 36m 32s


100%|██████████| 10000/10000 [37:23<00:00,  4.46it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9378
KNN iter training complete in 37m 23s


100%|██████████| 10000/10000 [38:32<00:00,  4.33it/s]


[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9307
KNN iter training complete in 38m 32s


100%|██████████| 10000/10000 [39:28<00:00,  4.22it/s]

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9]
acc 0.9146
KNN iter training complete in 39m 29s
